In [23]:
import pandas as pd
import time as time
import requests

In [ ]:
# This function scrapes tot_posts from Reddit including [removed] and [deleted] posts

def scrape_reddit(subreddit, tot_posts):
    
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()
    posts_scraped = 0

    while len(df) < tot_posts:
        if tot_posts < 100:
            params = {
                'subreddit': subreddit,
                'size': tot_posts,
                'before': int(time.time())
            }
            res = requests.get(url, params)
            data = res.json()
            posts = data['data']
            df = pd.DataFrame(posts)
        else:
            if len(df) == 0:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': int(time.time())
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df = pd.DataFrame(posts)
                posts_scraped += 100
            elif tot_posts - posts_scraped > 100:
                params = {
                    'subreddit': subreddit,
                    'size': 100,
                    'before': df.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
                posts_scraped += 100
            else:
                params = {
                    'subreddit': subreddit,
                    'size': (tot_posts - posts_scraped),
                    'before': df.iloc[-1]['created_utc']
                }
                res = requests.get(url, params)
                data = res.json()
                posts = data['data']
                df2 = pd.DataFrame(posts)
                df = df.append(df2, ignore_index=True)
        
    return df